# Running MD with ASE and metatensor

Now that we have a working model, let's use it to run some MD simulations! We'll use the MD engines provided within [ASE](https://wiki.fysik.dtu.dk/ase/), since it is easy to run directly inside a jupyter notebook.

In [1]:
import ase

import ase.md
import ase.io
import ase.units

import chemiscope

import metatomic.torch.ase_calculator


We'll start our simulation from the first conformer in our training set

In [4]:
atoms = ase.io.read("../propenol_conformers_dftb.xyz", "0")

chemiscope.show([atoms], mode="structure")


<StructureWidget(meta={'name': ' '}, structures=[{'size': 10, 'names': ['O', 'C', 'C', 'C', 'H', 'H', 'H', 'H'…

`metatomic.torch.ase_calculator.MetatomicCalculator` is an ASE `Calculator` that we can use to load our previously saved model and integrate it with ASE.

In [5]:
atoms.calc = metatomic.torch.ase_calculator.MetatomicCalculator(
    "propenol-model.pt",
    extensions_directory="extensions",
)

# we'll also need an integrator for the MD
integrator = ase.md.VelocityVerlet(atoms, timestep=1 * ase.units.fs)


Let's run the simulation for 100 steps, and keep the trajectory around for visualization.

| ![TASK](../img/clipboard.png) | Make a copy of the atoms after each simulation step, and put it in the `trajectory` |
|----------------------------|-------------------------------------------------------------------------------------|

In [6]:
trajectory = [atoms.copy()]

for _ in range(100):
    integrator.run(1)
    trajectory.append(atoms.copy())


We can now visualize our trajectory.

In [7]:
chemiscope.show(
    trajectory, mode="structure", settings={"structure": [{"playbackDelay": 50}]}
)


<StructureWidget(meta={'name': ' '}, settings={'structure': [{'playbackDelay': 50}]}, structures=[{'size': 10,…

Uh oh, it seems that our potential is not very stable …

This is to be expected here, since we trained a very simplified potential. There are
multiple ways to improve it:

- training on more data. This includes training with more representative structures, and
  training on both energies and forces simultaneously.
- training a more complex model. A first step could be to have different models 
  for different central atomic species.
- adding more physics inside the model

For this tutorial, we will start with the last option, demonstrating how
`metatomic` allow you to write your own Python code and use it in a simulation,
and how it allows you to combine multiple building blocks in a single model.

Here, one issue is that our model is not repulsive enough at short distances,
and the atoms end up very close to one another. To fix this, in the next
notebook we will add a Lennard-Jones baseline predictor to the model.